In [1]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb

data = pd.read_csv('big_data.csv', index_col=False)
data.head()

,0,1,2,3,4,5
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,Творчество поэтов Гумилева Ахматовой Мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1


Логрегрессию обучаем ледующим образом. Из вектора вопроса (q_vec) и векторов ответов (a1_vec, ... a4_vec) создаем четыре вектора q_vec | a1_vec), ... (q_vec | a4_vec) с соответствующей бинарной разметкой, где | - конкатенация. Затем каждый из таких сэмплов с соответствующей разметкой подаем на вход логрегрессии.

In [2]:
import pickle

with open('/Users/user/Python/Diploma/Embeddings/elmo_embed_big.pickle', 'rb') as f:
    elmo_embs = pickle.load(f)

elmo_embs = np.array(elmo_embs)

with open('/Users/user/Python/Diploma/Embeddings/fasttext_embed_big.pickle', 'rb') as f:
    fasttext_embs = pickle.load(f)

fasttext_embs = np.array(fasttext_embs)

with open('/Users/user/Python/Diploma/Embeddings/rusvec_embed_big.pickle', 'rb') as f:
    rusvec_embs = pickle.load(f)

rusvec_embs = np.array(rusvec_embs)

with open('/Users/user/Python/Diploma/Embeddings/bert_embed_big.pickle', 'rb') as f:
    bert_embs = pickle.load(f)

bert_embs = np.array(bert_embs)

In [3]:
embeddings = {'RusVectores': rusvec_embs, 'FastText': fasttext_embs, 'ELMO': elmo_embs, 'BERT': bert_embs}

In [13]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score, precision_score, f1_score

def get_samples(data, embeddings, train_part, random_seed):
    
    '''
    Из вектора вопроса (q_vec) и векторов ответов (a1_vec, ... a2_vec) 
    создаем четыре вектора (q_vec | a1_vec), ... (q_vec | a2_vec) 
    с соответствующей бинарной разметкой, где | - конкатенация. 
    
    Делим выборку на трейн и тест. 
    
    data - исходный датасет с разметкой
    embeddings - векторное представление исходного датасета
    train_part - доля тренировочной выборки
    random_seed - сид для повторяемости эксперимента
    
    return:
    
    X_train, X_test, y_train, y_test - разделенная на тренировочную и тестовую части выборка,
    имеющая вышеописанную структуру
    
    '''
    
    predata = np.zeros((embeddings.shape[0] * 4, embeddings.shape[2] * 2))
    target = []
    np.random.seed(random_seed)

    for i in range(len(data)):
        for j in range(1,5):
            cur_targ = 1 if j == int(data.iloc[i,5]) else 0
            cur_vec = np.hstack((embeddings[i,0], embeddings[i,j]))
            predata[i*4+j-1] = cur_vec
            target.append(cur_targ)
            
    all_inds = np.arange(embeddings.shape[0])
    train_size = int(len(all_inds) * train_part)
    train_inds = np.random.choice(all_inds, size=train_size, replace=False)
    test_inds = np.array(list(set(all_inds) - set(train_inds)))
    
    X_train, X_test, y_train, y_test = [],[],[],[]

    for i in range(len(train_inds)):
        for j in range(4):
            X_train.append(predata[train_inds[i] * 4 + j])
            y_train.append(target[train_inds[i] * 4 + j])

    for i in range(len(test_inds)):
        for j in range(4):
            X_test.append(predata[test_inds[i] * 4 + j])
            y_test.append(target[test_inds[i] * 4 + j])
            
    return X_train, X_test, y_train, y_test


def approach_logreg(data, embeddings, train_part=0.7, random_seed=42):
    
    '''
    Обучаем логрегрессию и делаем предсказание.
    
    data - исходный датасет с разметкой
    embeddings - векторное представление исходного датасета
    train_part - доля тренировочной выборки
    random_seed - сид для повторяемости эксперимента
    
    return:
    
    accuracy - значение метрики точности для предсказания
    
    '''
    
    X_train, X_test, y_train, y_test = get_samples(data, embeddings, train_part, random_seed)
    
    clf = LR(solver='lbfgs')
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict_proba(X_test)
    y_pred_new = []

    for i in range(len(y_pred) // 4):
        cur_probs, y_pred_splitted = [], [0,0,0,0]
        for j in range(4):
            cur_probs.append(y_pred[i*4 + j][1])
        y_pred_splitted[np.argmax(cur_probs)] = 1
        y_pred_new.extend(y_pred_splitted)
        
    y_pred_splitted, y_test_splitted = [], []

    for i in range(len(y_pred) // 4):
        y_pred_splitted.append(np.argmax(y_pred_new[i*4:i*4+4]))
        y_test_splitted.append(np.argmax(y_test[i*4:i*4+4]))
        
    accuracy = accuracy_score(y_test_splitted, y_pred_splitted)
    precision_macro = precision_score(y_test_splitted, y_pred_splitted, average='macro')
    precision_micro = precision_score(y_test_splitted, y_pred_splitted, average='micro')
    f1_macro = f1_score(y_test_splitted, y_pred_splitted, average='macro')
    f1_micro = f1_score(y_test_splitted, y_pred_splitted, average='micro')
    
    return accuracy, precision_macro, precision_micro, f1_macro, f1_micro

In [14]:
results_dict = {}
for key, embed in embeddings.items():
    results_dict[key] = approach_logreg(data, embed)

In [15]:
results = pd.DataFrame(data=results_dict, columns=list(embeddings.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.281010,0.313462,0.322115,0.335577
Precision Macro,0.280677,0.308265,0.317652,0.330577
Precision Micro,0.281010,0.313462,0.322115,0.335577
F1 Macro,0.279967,0.308299,0.317663,0.330363
F1 Micro,0.281010,0.313462,0.322115,0.335577
